In [9]:
import requests
from bs4 import BeautifulSoup


headers = {
    'User-Agent': (
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/72.0.3626.121 Safari/537.36'
    ),
}


def get_like_count(song_no_list):
    api_url = "https://www.melon.com/commonlike/getSongLike.json"
    params = {"contsIds": song_no_list}
    res = requests.get(api_url, params=params, headers=headers)
    res.raise_for_status()
    response = res.json()
    like_list = response["contsLike"]
    like_dict = {
        str(song["CONTSID"]): song["SUMMCNT"]
        for song in like_list}
    return like_dict


def get_song_list():
    res = requests.get("http://www.melon.com/chart/index.htm", headers=headers)
    res.raise_for_status()
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    tr_tag_list = soup.select('.d_song_list tbody tr')

    song_list = []

    for rank, tr_tag in enumerate(tr_tag_list, 1):
        song_no = tr_tag["data-song-no"]
        song_tag = tr_tag.select_one('a[href*=playSong]')
        album_tag = tr_tag.select_one('.wrap_song_info a[href*=goAlbumDetail]')
        artist_tag = tr_tag.select_one('a[href*=goArtistDetail]')
        
        song = {
            'song_no': song_no,
            'title': song_tag.text,
            'album': album_tag.text,
            'artist': artist_tag.text,
            'rank': rank,
        }
        song_list.append(song)

    song_no_list = [song["song_no"] for song in song_list]
    like_dict = get_like_count(song_no_list)

    for song in song_list:
        like_count = like_dict[song["song_no"]]
        song["like"] = like_count

    return song_list

In [10]:
melon_top100_list = get_song_list()
melon_top100 = get_song_list()

# f 객체에 대해서 슬라이싱 적용하기

In [55]:
from itertools import islice

with open("melon_top100.csv", "rt") as f:
    # TODO: 줄 단위로 읽어서 출력해보세요.

#     for index, line in islice(enumerate(f),10):   # 파일객체는 Iterable
#         print(f"[{index}]", line)
#         if index >= 10:
#             break
            
    for index, line in islice(enumerate(f),10):   # 파일객체는 Iterable
        print(f"[{index}]", line)

[0] ,song_no,title,album,artist,rank,like

[1] 0,33077590,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),쇼미더머니 9 Episode 1,미란이,1,96325

[2] 1,32872978,Dynamite,Dynamite (DayTime Version),방탄소년단,2,360973

[3] 2,33013877,잠이 오질 않네요,잠이 오질 않네요,장범준,3,82843

[4] 3,33077234,Life Goes On,BE,방탄소년단,4,151888

[5] 4,32998018,힘든 건 사랑이 아니다,힘든 건 사랑이 아니다,임창정,5,79274

[6] 5,32961718,Lovesick Girls,THE ALBUM,BLACKPINK,6,150487

[7] 6,32794652,취기를 빌려 (취향저격 그녀 X 산들),취기를 빌려 (취향저격 그녀 X 산들),산들,7,157119

[8] 7,33061995,밤하늘의 별을(2020),밤하늘의 별을(2020),경서,8,42768

[9] 8,33077591,Freak (Prod. Slom),쇼미더머니 9 Episode 1,릴보이 (lIlBOI),9,36493



# [(과제) CSV 파일을 생성하고, 엑셀에서 열어보기](https://pnu.nomade.kr/CSV-b531d9bdee33471c8da2bcd410448da4)

In [54]:
import csv
filename = os.path.basename("melon_top100_list.csv")
keys = melon_top100_list[0].keys()
with open(filename, 'w', encoding='utf-8-sig', newline='')  as f:
    dict_writer = csv.DictWriter(f, keys)
    dict_writer.writeheader()
    dict_writer.writerows(melon_top100_list)